# **PRO - Consolidados**

In [10]:
import pandas as pd
import numpy as np
import re

In [11]:
data_col = pd.read_csv(r"C:\Users\Hp\DENGUE\DATA\Data_Colombia.csv")
data_atl = pd.read_csv(r"C:\Users\Hp\DENGUE\DATA\Data_Atlantico.csv")
data_bqui= pd.read_csv(r"C:\Users\Hp\DENGUE\DATA\Data_Barranquilla.csv")

## Semana epidemiológica

In [12]:

def resumen_por_semana(df):
    # columnas a ignorar
    ignorar = ['edad_años', 'periodo', 'fecha_inicio_sintomas', 'fecha_consulta',
               'fecha_hospitalizacion', 'fecha_notificacion', 'fecha_nacimiento', 'año', 'semana',
               'direccion', 'localidad', 'barrio', 'periodo']
    
    # seleccionar solo columnas categóricas relevantes
    cols_cat = [c for c in df.columns if c not in ignorar]
    
    # combinar año y semana en una sola columna
    df['año_semana'] = df['año'].astype(str) + '_' + df['semana'].astype(str)
    
    resultados = []
    
    # para cada columna categórica (excepto año y semana)
    for col in cols_cat:
        if col not in ['año', 'semana']:
            conteo = df.groupby(['año_semana', col]).size().unstack(fill_value=0)
            resultados.append(conteo)
    
    # unir todos los conteos
    resumen = pd.concat(resultados, axis=1)
    
    # resetear el índice
    resumen = resumen.reset_index().fillna(0)
    resumen[['año', 'semana']] = resumen['año_semana'].str.split('_', expand=True).astype(int)
    resumen = resumen.sort_values(by=['año', 'semana']).reset_index(drop=True)
    resumen['año_semana'] = resumen['año'].astype(str) + '_' + resumen['semana'].astype(str)

    total_eventos = df.groupby('año_semana')['evento'].count().reset_index(name='total_casos')
    resumen = resumen.merge(total_eventos, on='año_semana', how='left')

    return resumen


In [13]:
consolidado_atl = resumen_por_semana(data_atl)
consolidado_col = resumen_por_semana(data_col)
consolidado_bquilla = resumen_por_semana(data_bqui)

In [32]:
consolidado_bquilla.to_csv("../DATA/Resumen/casosXsemana_bquilla.csv", index=False)
consolidado_atl.to_csv("../DATA/Resumen/casosXsemana_atl.csv", index=False)
consolidado_col.to_csv("../DATA/Resumen/casosXsemana_col.csv", index=False)

## Casos netos por unidad geográfica

### Nación

In [15]:
# Para data_col (solo departamento)
total_col = data_col.groupby('departamento').size().reset_index(name='total_casos')
print("Total de casos por departamento (data_col):")
total_col

Total de casos por departamento (data_col):


,departamento,total_casos
0,AMAZONAS,4870
1,ANTIOQUIA,71052
2,ARAUCA,8943
3,"ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANT...",954
4,ATLÁNTICO,55069
5,"BOGOTÁ, D.C.",316
6,BOLÍVAR,45869
7,BOYACÁ,6716
8,CALDAS,5161
9,CAQUETÁ,10403


### Departamento

In [16]:
# Para data_atl (solo municipio)
total_atl = data_atl.groupby('municipio').size().reset_index(name='total_casos')
print("\nTotal de casos por municipio (data_atl):")
total_atl


Total de casos por municipio (data_atl):


,municipio,total_casos
0,BARANOA,1206
1,CAMPO DE LA CRUZ,152
2,CANDELARIA,91
3,DESCONOCIDO,27
4,GALAPA,1143
5,JUAN DE ACOSTA,331
6,LURUACO,287
7,MALAMBO,2087
8,MANATÍ,264
9,PALMAR DE VARELA,187


In [33]:
total_col.to_csv("../DATA/Resumen/casosXdpt.csv", index=False)
total_atl.to_csv("../DATA/Resumen/casosXmun.csv", index=False)

## Consolidado por unidad geográfica, evento y año

### Nación

In [18]:

# Filtrar evento Clásico
data_clasico = data_col[data_col["evento"] == "Clasico"]

# Total por departamento
total_clasico = data_clasico.groupby("departamento").size().reset_index(name="total_casos_clasico")

# Total por municipio y año
total_clasico_año = data_clasico.groupby(["departamento", "año"]).size().reset_index(name="casos_por_año_clasico")

# Pivotear años como columnas
tabla_clasico_año = total_clasico_año.pivot(index="departamento", columns="año", values="casos_por_año_clasico").fillna(0).astype(int)
tabla_clasico_año["total_casos_clasico"] = tabla_clasico_año.sum(axis=1)
tabla_clasico_año.reset_index(inplace=True)

tabla_clasico_año

año,departamento,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,total_casos_clasico
0,AMAZONAS,274,485,160,115,62,48,1713,202,312,473,1004,4848
1,ANTIOQUIA,5761,6762,6779,27622,4088,3668,4978,2133,1238,2007,5239,70275
2,ARAUCA,1013,704,726,310,297,536,1954,346,262,1412,1203,8763
3,"ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANT...",67,99,153,68,23,16,28,69,189,109,123,944
4,ATLÁNTICO,7550,6523,4317,1361,1157,3245,3905,2613,7661,10143,5384,53859
5,"BOGOTÁ, D.C.",142,160,0,0,0,0,0,0,0,0,0,302
6,BOLÍVAR,4447,2864,3035,1204,479,1347,5238,2438,8139,6894,8610,44695
7,BOYACÁ,685,540,920,2076,195,91,829,505,287,192,331,6651
8,CALDAS,737,918,1144,781,102,46,431,493,74,180,178,5084
9,CAQUETÁ,1084,735,834,892,260,187,1200,1781,388,1042,1854,10257


In [20]:
# Filtrar evento Clásico
data_grave = data_col[data_col["evento"] == "Grave"]

# Total por departamento
total_grave = data_grave.groupby("departamento").size().reset_index(name="total_casos_grave")

# Total por departamento y año
total_grave_año = data_grave.groupby(["departamento", "año"]).size().reset_index(name="casos_por_año_grave")

# Pivotear años como columnas
tabla_grave_año = total_grave_año.pivot(index="departamento", columns="año", values="casos_por_año_grave").fillna(0).astype(int)
tabla_grave_año["total_casos_grave"] = tabla_grave_año.sum(axis=1)
tabla_grave_año.reset_index(inplace=True)

tabla_grave_año

año,departamento,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,total_casos_grave
0,AMAZONAS,5,5,1,0,0,0,4,0,4,1,2,22
1,ANTIOQUIA,125,177,64,111,22,57,41,25,23,51,81,777
2,ARAUCA,98,21,10,4,3,7,20,4,1,7,5,180
3,"ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANT...",2,4,1,0,0,0,0,2,0,0,1,10
4,ATLÁNTICO,145,127,93,50,39,47,58,43,188,333,87,1210
5,"BOGOTÁ, D.C.",6,8,0,0,0,0,0,0,0,0,0,14
6,BOLÍVAR,163,121,90,13,4,43,113,71,237,167,152,1174
7,BOYACÁ,14,7,15,9,3,5,5,0,0,3,4,65
8,CALDAS,21,22,16,1,1,1,5,6,0,2,2,77
9,CAQUETÁ,18,16,9,10,4,3,18,36,6,10,16,146


In [35]:
tabla_clasico_año.to_csv("../DATA/Resumen/casos_dpto_año_clasico.csv")
tabla_grave_año.to_csv("../DATA/Resumen/casos_dpto_año_grave.csv")

### Departamento

In [26]:
# Filtrar evento Clásico
data_clasico = data_atl[data_atl["evento"] == "Clasico"]

# Total por municipio
total_clasico = data_clasico.groupby("municipio").size().reset_index(name="total_casos_clasico")

# Total por municipio y año
total_clasico_año = data_clasico.groupby(["municipio", "año"]).size().reset_index(name="casos_por_año_clasico")

# Pivotear años como columnas
tabla_clasico_año = total_clasico_año.pivot(index="municipio", columns="año", values="casos_por_año_clasico").fillna(0).astype(int)
tabla_clasico_año["total_casos_clasico"] = tabla_clasico_año.sum(axis=1)
tabla_clasico_año.reset_index(inplace=True)

tabla_clasico_año

año,municipio,2017,2018,2019,2020,2021,2022,2023,total_casos_clasico
0,BARANOA,7,158,387,167,105,95,267,1186
1,CAMPO DE LA CRUZ,4,0,29,6,12,76,18,145
2,CANDELARIA,3,1,28,2,1,11,42,88
3,DESCONOCIDO,1,20,3,0,0,0,2,26
4,GALAPA,38,134,61,35,160,342,339,1109
5,JUAN DE ACOSTA,3,49,34,9,27,94,109,325
6,LURUACO,13,5,47,31,166,10,9,281
7,MALAMBO,22,154,159,216,717,231,546,2045
8,MANATÍ,4,4,48,6,48,131,15,256
9,PALMAR DE VARELA,10,6,22,16,8,15,106,183


In [27]:
# Filtrar evento Clásico
data_grave = data_atl[data_atl["evento"] == "Grave"]

# Total por municipio
total_grave = data_grave.groupby("municipio").size().reset_index(name="total_casos_grave")

# Total por municipio y año
total_grave_año = data_grave.groupby(["municipio", "año"]).size().reset_index(name="casos_por_año_grave")

# Pivotear años como columnas
tabla_grave_año = total_grave_año.pivot(index="municipio", columns="año", values="casos_por_año_grave").fillna(0).astype(int)
tabla_grave_año["total_casos_grave"] = tabla_grave_año.sum(axis=1)
tabla_grave_año.reset_index(inplace=True)

tabla_grave_año

año,municipio,2017,2018,2019,2020,2021,2022,2023,total_casos_grave
0,BARANOA,0,4,8,3,3,1,1,20
1,CAMPO DE LA CRUZ,0,0,1,0,3,3,0,7
2,CANDELARIA,0,0,0,0,0,2,1,3
3,DESCONOCIDO,1,0,0,0,0,0,0,1
4,GALAPA,1,2,1,0,6,20,4,34
5,JUAN DE ACOSTA,0,0,0,0,0,5,1,6
6,LURUACO,0,0,2,0,2,2,0,6
7,MALAMBO,0,3,5,5,13,14,2,42
8,MANATÍ,0,0,1,0,0,6,1,8
9,PALMAR DE VARELA,0,0,1,0,0,1,2,4


In [34]:
tabla_clasico_año.to_csv("../DATA/Resumen/casos_mun_año_clasico.csv")
tabla_grave_año.to_csv("../DATA/Resumen/casos_mun_año_grave.csv")